##Prepare meteorological data for WOFOST run on gridded data

This notebook reads COSMO climate data [Tmin, Tmax](scClim climate simulations) and agricultural exposure data (or the spatial_units.gpkg file) as described in [Portmann et al. 2023](https://egusphere.copernicus.org/preprints/2023/egusphere-2023-2598/) and produces one .csv file for each meteorological variable with time as rows and grid cell IDs (of all 1x1km grid cells with exposure) as columns.

In [20]:
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import os

First, define directories of gridded exposure data (polygons), the MeteoSwiss data and the directory to store the output data

In [21]:
#INPUT DATA
which='PGW'
datadir_COSMO=f'C:/Users/F80840370/projects/scClim/wofost/COSMO_TminTmax/{which}/daily_t2m_maxmin/T2M_{which}_daily_CH.nc'
datadir_exposure='C:/Users/F80840370/projects/scClim/climada/data/scClim/exposure/GIS/Weizen_Mais_Raps_Gerste_polygons.gpkg'

#OUTPUT DATA
OUTDIR='C:/Users/F80840370/projects/scClim/wofost/winter_wheat_phenology/meteo_data/ClimateSimulations/'

variables=['TMIN_2M','TMAX_2M']


Create spatial units file from exposure or just read it if it is there

In [23]:
#create from exposure
units=gpd.read_file(datadir_exposure)
units=units[units.n_fields>0]
units_lv95=units.to_crs(crs=2056)
units_lv95['X']=np.round(units_lv95.centroid.x)
units_lv95['Y']=np.round(units_lv95.centroid.y)
units_lv95['ID']=units_lv95.index


#save
#nits_lv95.to_file("spatial_units.gpkg", driver="GPKG")
#read
#units_lv95=gpd.read_file("spatial_units.gpkg", driver="GPKG")

In [24]:
units_new = gpd.GeoDataFrame.copy(units_lv95)
units_new['geometry']=units_new.geometry.centroid
units_new=units_new.to_crs(epsg=4326)
units_lv95['latitude']=units_new.geometry.y.values
units_lv95['longitude']=units_new.geometry.x.values

In [25]:
units_lv95.latitude

147475    46.181038
147476    46.190032
147477    46.199026
148112    46.154227
148116    46.190203
            ...    
351650    46.295079
351651    46.304069
354204    46.239913
363869    46.819433
365790    46.827412
Name: latitude, Length: 12742, dtype: float64

In [26]:
'n_fields' in units.columns

True

Read gridded COSMO climate data. Make sure to use .load() to load the data into memory. Otherwise subsequent computations may take a long time. This step takes quite some time (up to 1h is to be expected). Change coordinates to lat lon.

In [27]:
#get list of datafiles #1961-2022
cosmo_data= xr.open_dataset(datadir_COSMO)
cosmo_data.load()
lon=xr.DataArray(cosmo_data.lon.values[0,0,:],coords={'rlon':cosmo_data.rlon},dims='rlon')
lat=xr.DataArray(cosmo_data.lat.values[0,:,0],coords={'rlat':cosmo_data.rlat},dims='rlat')
cosmo_data=cosmo_data.assign_coords({'lon2':lon, 'lat2':lat})
cosmo_data=cosmo_data.swap_dims({'rlon': 'lon2', 'rlat': 'lat2'})


In [28]:
cosmo_data

<xarray.Dataset>
Dimensions:       (time: 4018, bnds: 2, lat2: 129, lon2: 168)
Coordinates:
  * time          (time) datetime64[ns] 2085-01-01 2085-01-02 ... 2086-10-19
    lon           (time, lat2, lon2) float32 5.82 5.848 5.877 ... 10.57 10.6
    lat           (time, lat2, lon2) float32 45.3 45.3 45.3 ... 47.94 47.94
    rlon          (lon2) float64 -2.94 -2.92 -2.9 -2.88 ... 0.34 0.36 0.38 0.4
    rlat          (lat2) float64 -1.62 -1.6 -1.58 -1.56 ... 0.88 0.9 0.92 0.94
    height_2m     (time) float32 2.0 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0 2.0
  * lon2          (lon2) float32 5.82 5.848 5.877 5.905 ... 10.51 10.54 10.57
  * lat2          (lat2) float32 45.3 45.32 45.34 45.36 ... 47.82 47.84 47.86
Dimensions without coordinates: bnds
Data variables:
    time_bnds     (time, bnds) datetime64[ns] 2084-12-31 ... 2086-10-19
    rotated_pole  (time) int32 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    TMIN_2M       (time, lat2, lon2) float32 275.9 274.9 274.3 ... 278.8 278.7
    TMAX_2M       (time, lat2, lon2) float32 279.7 278.4 277.5 ... 285.6 286.0
Attributes:
    CDI:             Climate Data Interface version 1.9.10 (https://mpimet.mp...
    Conventions:     CF-1.4
    source:          COSMO
    title:           COSMO driven by ERA5
    conventionsURL:  http://www.cfconventions.org/
    creation_date:   2023-08-19 15:28:31
    history:         Fri Feb 02 19:26:36 2024: cdo -L -selindexbox,797,964,66...
    CDO:             Climate Data Operators version 1.9.10 (https://mpimet.mp...

Create for each variable a DataFrame with time as column and spatial units (ID) as rows. Write one file per variable for all years

In [29]:
datalists={var: [pd.DataFrame({'time': cosmo_data.time.values})] for var in variables}

for ind in units_lv95.index:
      LON=units_lv95.loc[ind].longitude
      LAT=units_lv95.loc[ind].latitude
      ID=units_lv95.loc[ind].ID
      data=cosmo_data.sel(lon2=LON,lat2=LAT,method='nearest')
      for var in variables:
        datalists[var].append(pd.DataFrame({ID: data[var].values}))

print('now save....')

for var in variables:
      df_out=pd.concat(datalists[var],axis=1)
      filename='{}_{}'.format(var,which)
      df_out.to_csv(OUTDIR+filename+'.csv')


now save....


In [ ]:
units_lv95.loc[ind]

n_fields                                                   1.0
area_ha                                               0.308657
geometry     POLYGON ((2825000.0007423838 1190000.001071762...
X                                                    2825500.0
Y                                                    1190500.0
ID                                                      365790
latitude                                             46.827412
longitude                                            10.394723
Name: 365790, dtype: object

Voilá, you are ready to run wofost